#En este notebook veremos la aplicación de PCA a un set de datos

###Cargamos las librerias necesarias

In [1]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Preprocesado y modelado
# ==============================================================================
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

###Cargamos el data set.

El set de datos USArrests contiene el porcentaje de asaltos (Assault), asesinatos (Murder) y secuestros (Rape) por cada 100,000 habitantes para cada uno de los 50 estados de USA (1973). Además, también incluye el porcentaje de la población de cada estado que vive en zonas rurales (UrbanPoP).

In [2]:
USArrests = sm.datasets.get_rdataset("USArrests", "datasets")
datos = USArrests.data
datos.head()

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


###Vemos el tipo de datos. 

Todo se ve en orden ya que son numéricos

In [3]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, Alabama to Wyoming
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Murder    50 non-null     float64
 1   Assault   50 non-null     int64  
 2   UrbanPop  50 non-null     int64  
 3   Rape      50 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 2.0+ KB


In [4]:
datos.describe()

,Murder,Assault,UrbanPop,Rape
count,50.00000,50.000000,50.000000,50.000000
mean,7.78800,170.760000,65.540000,21.232000
std,4.35551,83.337661,14.474763,9.366385
min,0.80000,45.000000,32.000000,7.300000
25%,4.07500,109.000000,54.500000,15.075000
50%,7.25000,159.000000,66.000000,20.100000
75%,11.25000,249.000000,77.750000,26.175000
max,17.40000,337.000000,91.000000,46.000000


### Vemos que la escala de los datos es muy diferente. Es necesario escalar los datos. Se deben dejar con media cero y desviación estándar 1.

###Por defecto, PCA() centra los valores pero no los escala. Esto es importante ya que, si las variables tienen distinta dispersión, como en este caso, es necesario escalarlas. 

###Se escalan los datos

In [5]:
# Se crea primero el objeto Standard Scaler
scaler = StandardScaler()

# Luego "entrenamos" con los datos para obtener los parámetros del escalamiento
scaler.fit(datos)

# Finalmente aplicamos el escalamiento
array_scaler = scaler.transform(datos)

In [6]:
#Corroboramos que dio resultado
pd.DataFrame(array_scaler).describe()

,0,1,2,3
count,5.000000e+01,5.000000e+01,5.000000e+01,5.000000e+01
mean,-7.105427e-17,1.387779e-16,-4.396483e-16,8.593126e-16
std,1.010153e+00,1.010153e+00,1.010153e+00,1.010153e+00
min,-1.620693e+00,-1.524362e+00,-2.340661e+00,-1.502548e+00
25%,-8.611383e-01,-7.486054e-01,-7.704502e-01,-6.640245e-01
50%,-1.247758e-01,-1.425453e-01,3.210209e-02,-1.220847e-01
75%,8.029251e-01,9.483628e-01,8.521012e-01,5.330962e-01
max,2.229265e+00,2.015028e+00,1.776781e+00,2.671197e+00


###Aplicamos PCA

[Acá](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) puede encontrar información de la función PCA

In [7]:
# Al no incluir el número de componentes, lo que hace PCA es conservarlos todos
pca = PCA()

In [8]:
pca.fit(array_scaler)

PCA()

###Un poco de interpretación:

Una vez entrenado el objeto PCA, pude accederse a toda la información de las componentes creadas.

components_ contiene el valor que definen cada componente (eigenvector). Las filas se corresponden con las componentes principales (ordenadas de mayor a menor varianza explicada). Las filas se corresponden con las variables de entrada.

In [9]:
# Se combierte el array a dataframe para añadir nombres a los ejes.
pd.DataFrame(
    data    = pca.components_,
    columns = datos.columns,
    index   = ['PC1', 'PC2', 'PC3', 'PC4']
)

,Murder,Assault,UrbanPop,Rape
PC1,0.535899,0.583184,0.278191,0.543432
PC2,0.418181,0.187986,-0.872806,-0.167319
PC3,-0.341233,-0.268148,-0.378016,0.817778
PC4,0.649228,-0.743407,0.133878,0.089024


### Analizar con detalle el vector que forma cada componente puede ayudar a interpretar qué tipo de información recoge cada una de ellas. Por ejemplo, la primera componente es el resultado de la siguiente combinación lineal de las variables originales:

**PC1=0.535899 Murder+0.583184 Assault+0.278191 UrbanPop+0.543432 Rape**


###Los pesos asignados en la primera componente a las variables Assault, Murder y Rape son aproximadamente iguales entre ellos y superiores al asignado a UrbanPoP. Esto significa que la primera componente recoge mayoritariamente la información correspondiente a los delitos. En la segunda componente, es la variable UrbanPoP es la que tiene el peso mayor (aunque sea negativo), por lo que se corresponde principalmente con el nivel de urbanización del estado. Si bien en este ejemplo la interpretación de las componentes es bastante clara, no en todos los casos ocurre lo mismo, sobre todo a medida que aumenta el número de variables.

###Una vez calculadas las componentes principales, se puede conocer la varianza explicada por cada una de ellas, la proporción respecto al total y la proporción de varianza acumulada. Esta información está almacenada en los atributos **explained_variance_** y **explained_variance_ratio_** del modelo.

In [10]:
print(pca.explained_variance_ratio_)

[0.62006039 0.24744129 0.0891408  0.04335752]


###En orden, PC1 explica la varianza total en un 62%

Si sumamos todo, debe dar un 100% (o casi un 100%)

In [11]:
print(sum(pca.explained_variance_ratio_)*100)

99.99999999999999


###Podemos obtener la varianza acumulada simplemente sumando y acumulando lo aportado por cada componente.

In [12]:
pca.explained_variance_ratio_.cumsum()

array([0.62006039, 0.86750168, 0.95664248, 1.        ])

###Esto nos dice por ejemplo, que solo necesitamos 2 dimensiones (en el nuevo espacio de coordenadas) para explicar el 86.8% de la varianza total.

###Una vez hecho esto, podemos transformar los datos originales al espacio nuevo creado.

La transformación es el resultado de multiplicar los vectores que definen cada componente con el valor de las variables. 

In [13]:
# Aplicamos la transformación
proyecciones = pca.transform(datos)
# Lo transformamos a DataFrame, ya que queda como arreglo.
proyecciones = pd.DataFrame(proyecciones,columns = ['PC1', 'PC2', 'PC3', 'PC4'], index = datos.index)
proyecciones.head()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but PCA was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


,PC1,PC2,PC3,PC4
Alabama,172.361042,-4.285324,-72.375325,-157.222134
Alaska,196.272181,4.281646,-55.689005,-178.636176
Arizona,214.898439,-16.356340,-86.489771,-199.833081
Arkansas,140.027276,-7.505767,-56.905170,-127.104356
California,213.160491,-30.570846,-78.277715,-183.540153


###Comparemos con el original

In [14]:
datos.head()

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6


###Acá podemos decidir con cuantos componentes nos quedamos y así reducimos la dimensión. Eso dependiendo de con cuanta varianza explicada queremos quedarnos.

###Finalmente, se puede reconstruir al espacio original una vez estando en el espacio transformado. Esto se hace con el método **inverse_transform()**. Es importante tener en cuenta que, la reconstrucción, solo será completa si se han incluido todas las componentes.

In [ ]:
recostruccion = pca.inverse_transform(proyecciones)
recostruccion = pd.DataFrame(recostruccion, columns = datos.columns, index = datos.index)
recostruccion.head()

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236.0,58.0,21.2
Alaska,10.0,263.0,48.0,44.5
Arizona,8.1,294.0,80.0,31.0
Arkansas,8.8,190.0,50.0,19.5
California,9.0,276.0,91.0,40.6


#Más adelante en el curso, veremos otra aplicación de este tipo de técnicas de reduccióón de dimensionalidad. 


#**SPOILER: Imágenes**

In [16]:
datos

,Murder,Assault,UrbanPop,Rape
Alabama,13.2,236,58,21.2
Alaska,10.0,263,48,44.5
Arizona,8.1,294,80,31.0
Arkansas,8.8,190,50,19.5
California,9.0,276,91,40.6
Colorado,7.9,204,78,38.7
Connecticut,3.3,110,77,11.1
Delaware,5.9,238,72,15.8
Florida,15.4,335,80,31.9
Georgia,17.4,211,60,25.8


In [17]:
pca = PCA(0.85)
pca.fit(array_scaler)
proyecciones = pca.transform(datos)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but PCA was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [19]:
proyecciones

array([[172.36104197,  -4.28532425],
       [196.27218078,   4.28164602],
       [214.89843921, -16.35634049],
       [140.02727553,  -7.50576662],
       [213.16049101, -30.57084561],
       [165.93277753, -32.90142215],
       [ 93.37146167, -47.0049004 ],
       [170.57548203, -18.27783943],
       [243.2101233 ,  -5.74679717],
       [163.08839827,  -9.74388283],
       [ 63.73388526, -64.95905408],
       [ 94.11441775, -25.86191629],
       [186.91829242, -25.30106482],
       [ 99.25270773, -35.99281838],
       [ 55.83492489, -40.19346184],
       [ 98.42389024, -36.4895145 ],
       [ 92.08910968, -23.56643054],
       [183.89036716,  -8.07128167],
       [ 67.95613551, -29.33721623],
       [214.75695528,  -2.00834795],
       [121.75648672, -47.06596933],
       [194.85680168, -17.46422484],
       [ 69.89388618, -45.43420457],
       [181.20563024,  14.15636227],
       [143.42792849, -28.58975369],
       [ 90.43881571, -26.0032378 ],
       [ 88.00356224, -35.9020321 ],
 

In [20]:
reconstruccion = pca.inverse_transform(proyecciones)
reconstruccion

array([[ 90.57615129,  99.71255971,  51.68952656,  94.38353613],
       [106.97266107, 115.26761164,  50.86408247, 105.94420252],
       [108.32405212, 122.25049637,  74.05870003, 119.51942884],
       [ 71.90177545,  80.25063945,  45.50538984,  77.35116983],
       [101.44845254, 118.56483108,  85.9817268 , 120.95332361],
       [ 75.16454317,  90.58428663,  74.87754952,  95.67821739],
       [ 30.38116736,  45.61646381,  67.00125676,  58.60584449],
       [ 83.76786855,  96.04085895,  63.40555398,  95.7544141 ],
       [127.93297676, 140.75584861,  72.67467708, 133.12973222],
       [ 83.32428165,  93.27877521,  53.87422543,  90.25780254],
       [  6.9903222 ,  24.95719184,  74.42684998,  45.50389885],
       [ 39.62090852,  50.02432028,  48.75421289,  55.47197542],
       [ 89.58899358, 104.25145325,  74.08188932, 105.81073822],
       [ 38.13796602,  51.11642316,  59.02595236,  59.9593758 ],
       [ 13.11377027,  25.00622224,  50.61386901,  37.06760519],
       [ 37.48609435,  50

In [21]:
pca = PCA(0.99)
pca.fit(array_scaler)
proyecciones = pca.transform(datos)
proyecciones

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but PCA was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


array([[ 172.36104197,   -4.28532425,  -72.37532533, -157.22213422],
       [ 196.27218078,    4.28164602,  -55.68900498, -178.63617574],
       [ 214.89843921,  -16.35634049,  -86.48977119, -199.83308142],
       [ 140.02727553,   -7.50576662,  -56.90516975, -127.10435568],
       [ 213.16049101,  -30.57084561,  -78.27771475, -183.54015322],
       [ 165.93277753,  -32.90142215,  -55.23524466, -132.63852196],
       [  93.37146167,  -47.0049004 ,  -50.65227636,  -68.33561578],
       [ 170.57548203,  -18.27783943,  -80.12884508, -162.05475526],
       [ 243.2101233 ,   -5.74679717,  -99.23885553, -225.49330323],
       [ 163.08839827,   -9.74388283,  -64.09904531, -135.2329231 ],
       [  63.73388526,  -64.95905408,  -28.99955823,  -17.84569374],
       [  94.11441775,  -25.86191629,  -41.86542297,  -79.02736245],
       [ 186.91829242,  -25.30106482,  -82.06641994, -165.10805794],
       [  99.25270773,  -35.99281838,  -40.15533848,  -68.75904176],
       [  55.83492489,  -40.193461

In [22]:
reconstruccion = pca.inverse_transform(proyecciones)
reconstruccion

array([[ 13.2, 236. ,  58. ,  21.2],
       [ 10. , 263. ,  48. ,  44.5],
       [  8.1, 294. ,  80. ,  31. ],
       [  8.8, 190. ,  50. ,  19.5],
       [  9. , 276. ,  91. ,  40.6],
       [  7.9, 204. ,  78. ,  38.7],
       [  3.3, 110. ,  77. ,  11.1],
       [  5.9, 238. ,  72. ,  15.8],
       [ 15.4, 335. ,  80. ,  31.9],
       [ 17.4, 211. ,  60. ,  25.8],
       [  5.3,  46. ,  83. ,  20.2],
       [  2.6, 120. ,  54. ,  14.2],
       [ 10.4, 249. ,  83. ,  24. ],
       [  7.2, 113. ,  65. ,  21. ],
       [  2.2,  56. ,  57. ,  11.3],
       [  6. , 115. ,  66. ,  18. ],
       [  9.7, 109. ,  52. ,  16.3],
       [ 15.4, 249. ,  66. ,  22.2],
       [  2.1,  83. ,  51. ,   7.8],
       [ 11.3, 300. ,  67. ,  27.8],
       [  4.4, 149. ,  85. ,  16.3],
       [ 12.1, 255. ,  74. ,  35.1],
       [  2.7,  72. ,  66. ,  14.9],
       [ 16.1, 259. ,  44. ,  17.1],
       [  9. , 178. ,  70. ,  28.2],
       [  6. , 109. ,  53. ,  16.4],
       [  4.3, 102. ,  62. ,  16.5],
 